YOLO V3
===

# 1.概述

YOLO算法创新性地提出了将输入图片进行N*N的栅格化（每个小单元叫grid cell），然后将图片中某个对象的位置的预测任务交与该对象中心位置所在的grid cell的bouding box。简单理解的话，可以认为这也是一种很粗糙的区域推荐（region proposal），在训练的时候，我们通过grid cell的方式告诉模型，图片中对象A应该是由中心落在特定grid cell 的某个范围内的某些像素组成，模型接收到这些信息后就在grid cell周围以一定大小范围去寻找所有满足对象A特征的像素，经过很多次带惩罚的尝试训练后，它就能找到这个准确的范围了（说明不是瞎找，如滑动窗口），当然这个方位不仅是指长宽的大小范围，也包括小幅度的中心位置坐标变化，但是不管怎么变，中心位置不能越过该grid cell的范围。这大大限制了模型在图片中瞎找时做的无用功。这样将位置检测和类别识别结合到一个CNN网络中预测，即只需要扫描一遍（you only look once）图片就能推理出图片中所有对象的位置信息和类别。举例如下图。

![Images](images/02_04_001.jpg)

不管是YOLOv1还是v2、v3，其主要的核心还是以上所述，只是在bounding box的拟合方式、骨干网络的设计、模型训练的稳定性、精度方面有所提升罢了

# 2.训练
YOLO算法几乎是输入原图就直接预测出每个grid cell“附近”是否有某个对象和具体的box位置，那最终这个想法数学化后便体现在loss函数上。在提出loss函数之前要先了解三个概念
- anchor box
- 置信度(confidence)
- 对象条件类别概率(conditional class probabilities)

在网络最后的输出中，对于每个grid cell对应bounding box的输出有三类参数
- 对象的box参数，一共是四个值，即box的中心点坐标（x,y）和box的宽和高（w,h）
- 置信度，这是个区间在\[0,1\]之间的值
- 一组条件类别概率，都是区间在\[0,1\]之间的值，代表概率。